<a href="https://colab.research.google.com/github/JafarLone/RealTime-Emotion-Sentiment-Detector/blob/main/s_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create the project structure with the 'assets' directory for the background image
!mkdir -p my_project/assets


In [ ]:
# Upload the background image
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload the image from your local machine


Saving background.webp to background.webp


In [ ]:
# Move the uploaded image to the assets folder (adjust the name if it's different)
!mv background.webp my_project/assets/background.png


In [ ]:
# Verify the image exists in the assets folder
!ls my_project/assets/


background.png


In [ ]:
!pip install streamlit opencv-python-headless deepface mediapipe


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=cdbd491a1de3e

In [ ]:
!npm install -g localtunnel
!pip install google-cloud-speech
!pip install pydub



added 22 packages, and audited 23 packages in 4s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.9/305.9 kB 4.5 MB/s eta 0:00:00


In [ ]:
!apt-get install -y ffmpeg  # Install ffmpeg binary
!pip install ffmpeg-python  # Install ffmpeg-python library


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!pip install SpeechRecognition


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 15.7 MB/s eta 0:00:00


In [ ]:
%%writefile /content/my_project/app.py

import cv2
from deepface import DeepFace
import streamlit as st
import tempfile
import time
from pydub import AudioSegment
import speech_recognition as sr
from transformers import pipeline
import ffmpeg
import moviepy.editor as mp
import base64

# Initialize the sentiment analysis model
sentiment_analysis = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Function to set background image from local file
def add_bg_from_local(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    st.markdown(
        f"""
        <style>
        .stApp {{
            background-image: url("data:image/png;base64,{encoded_string}");
            background-size: cover;
            background-repeat: no-repeat;
            background-attachment: fixed;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

# Set background using local image
add_bg_from_local('/content/my_project/assets/background.png')

# Custom CSS styling for sidebar and main content area
st.markdown("""
    <style>
    .sidebar .sidebar-content {
        background-color: #1e1e2d;
        padding: 20px;
    }
    .sidebar .sidebar-content h2, .sidebar .sidebar-content label {
        color: #FFFFFF;
        font-size: 1.5em;
        font-weight: 600;
    }
    .sidebar .sidebar-content .stRadio, .sidebar .sidebar-content .stFileUploader label {
        font-size: 1.2em;
        color: #FFFFFF;
    }
    .sidebar .sidebar-content .stFileUploader {
        background-color: #29293d;
        border-radius: 5px;
        padding: 10px;
        text-align: center;
    }

    .main-title {
        font-size: 2.8em;
        font-weight: bold;
        color: purple;
        text-align: center;
        padding-top: 20px;
        padding-bottom: 20px;
        font-family: Arial, sans-serif;
    }
    </style>
""", unsafe_allow_html=True)

# Title section in the main area
st.markdown("<h1 class='main-title'>Emotion & Sentiment AI</h1>", unsafe_allow_html=True)

# Sidebar for video upload options
st.sidebar.header("Upload or Capture Video")
st.sidebar.radio("Choose an option:", ["Upload Video", "Capture Live Stream"])
st.sidebar.write("Max file size: 200 MB")
uploaded_file = st.sidebar.file_uploader(
    "Upload a video", type=["mp4", "mov", "avi"], help="Drag and drop or browse to upload your video."
)

# Display main banner text
st.markdown(
    """
    <div style="text-align: center;">
        <h2>Emotion & Sentiment AI</h2>
    </div>
    """,
    unsafe_allow_html=True
)

if uploaded_file is not None:
    # Save the uploaded video to a temporary file
    tfile = tempfile.NamedTemporaryFile(delete=False)
    tfile.write(uploaded_file.read())

    # Display the original video (unprocessed)
    st.video(tfile.name)
    st.write("Original video displayed above.")

    # "Analyze Emotion & Sentiment" button
    if st.button("Analyze Emotion & Sentiment"):
        sentiments = []

        # Check for an audio stream
        try:
            probe = ffmpeg.probe(tfile.name)
            audio_streams = [stream for stream in probe['streams'] if stream['codec_type'] == 'audio']

            if audio_streams:
                # Extract audio from video
                audio_path = tempfile.mktemp(suffix='.wav')
                video = AudioSegment.from_file(tfile.name)
                video.export(audio_path, format="wav")

                recognizer = sr.Recognizer()
                audio_file = sr.AudioFile(audio_path)
                segment_duration = 7
                segment_number = 1

                with audio_file as source:
                    recognizer.adjust_for_ambient_noise(source)
                    while True:
                        try:
                            audio_data = recognizer.record(source, duration=segment_duration)
                            if not audio_data.frame_data:
                                break

                            try:
                                text = recognizer.recognize_google(audio_data)
                                st.write(f"Segment {segment_number} Text:", text)
                                sentiment = sentiment_analysis(text[:512])
                                sentiment_label = sentiment[0]['label'] if sentiment else "No sentiment detected"
                                sentiments.append((segment_number, text, sentiment_label))

                            except sr.UnknownValueError:
                                sentiments.append((segment_number, "Unintelligible", "No sentiment detected"))

                            segment_number += 1

                        except Exception as e:
                            st.write("Error in processing audio segment:", e)
                            break
            else:
                st.write("Warning: The uploaded video file has no audio stream. Sentiment analysis will not be performed.")

        except Exception as e:
            st.write("Error during audio stream check:", e)

        # Video processing with OpenCV
        cap = cv2.VideoCapture(tfile.name)
        frame_width, frame_height, fps = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), cap.get(cv2.CAP_PROP_FPS)
        fps, frame_width, frame_height = fps or 20, frame_width or 640, frame_height or 480

        frames, frame_skip, frame_count = [], 5, 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % frame_skip == 0:
                small_frame = cv2.resize(frame, (320, 240))

                try:
                    result = DeepFace.analyze(small_frame, actions=['emotion'], enforce_detection=False)
                    emotion = result[0]['dominant_emotion']
                    face_areas = result[0]['region']
                    x, y, w, h = (int(face_areas['x'] * frame_width / 320),
                                  int(face_areas['y'] * frame_height / 240),
                                  int(face_areas['w'] * frame_width / 320),
                                  int(face_areas['h'] * frame_height / 240))
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, f"Emotion: {emotion}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                except Exception as e:
                    st.write("Error detecting emotion:", e)

            frames.append(frame)
            frame_count += 1

        cap.release()

        output_path = '/content/my_project/processed_video.mp4'
        video_clip = mp.ImageSequenceClip([cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) for frame in frames], fps=fps)
        video_clip.write_videofile(output_path, codec="libx264")

        st.write("Processed video displayed below:")
        st.video(output_path)

        st.write("Sentiment Analysis Results:")
        if sentiments:
            for segment_num, text, sentiment in sentiments:
                st.write(f"Segment {segment_num}:")
                st.write(f" - Text: {text}")
                st.write(f" - Sentiment: {sentiment}")
        else:
            st.write("No sentiments detected or no audio available.")


Overwriting /content/my_project/app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com


34.80.47.107


In [ ]:
# Run the app and open it using Localtunnel
!streamlit run my_project/app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.47.107:8501

your url is: https://tired-ravens-add.loca.lt
2024-11-06 14:27:00.666613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 14:27:00.696063: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 14:27:00.704915: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 14:27:00.725758: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instru